In [1]:
using Yao
using Statistics: mean
using LinearAlgebra

In [2]:
rotor(noleading::Bool=false, notrailing::Bool=false) = noleading ? (notrailing ? Rx(0) : chain(Rx(0), Rz(0))) : (notrailing ? chain(Rz(0), Rx(0)) : chain(Rz(0), Rx(0), Rz(0)))

function twoqubit_circuit(nlayer::Int, nrepeat::Int)
    nbit_measure = nbit_virtual = 1
    nbit_used = nbit_measure + nbit_virtual
    circuit = chain(nbit_used)

    for i=1:nrepeat
        unit = chain(nbit_used)
        for j=1:nlayer
            push!(unit, put(nbit_used, 1=>rotor(true, false)))
            push!(unit, control(nbit_used, 1, 2=>(j%2==1 ? X : Z)))
            j == nlayer && push!(unit, put(nbit_used, 1=>rotor(false, true)))
        end
        push!(unit, Measure{nbit_used, 1, AbstractBlock}(Z, (1,), 0, false))
        if i==nrepeat
            for k=2:nbit_used
                push!(unit, Measure{nbit_used, 1, AbstractBlock}(Z, (k,), 0, false))
            end
        end
        push!(circuit, unit)
    end
    dispatch!(circuit, :random)
end

twoqubit_circuit (generic function with 1 method)

In [3]:
circuit = twoqubit_circuit(2, 3)

nqubits: 2, datatype: Complex{Float64}
chain
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 0.84570445572993)
│  │     └─ rot(Z gate, 0.4318469396717437)
│  ├─ control(1)
│  │  └─ (2,)X gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 0.6605822615388279)
│  │     └─ rot(Z gate, 0.22101235663174257)
│  ├─ control(1)
│  │  └─ (2,)Z gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(Z gate, 0.49914112233806573)
│  │     └─ rot(X gate, 0.04810499026441151)
│  └─ Measure{2,1,AbstractBlock}
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 0.28418220191407983)
│  │     └─ rot(Z gate, 0.9350756615919091)
│  ├─ control(1)
│  │  └─ (2,)X gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 0.9010875863516428)
│  │     └─ rot(Z gate, 0.7208934347561435)
│  ├─ control(1)
│  │  └─ (2,)Z gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(Z gate, 0.6851329727888991)
│  │     └─ rot(X gate, 0.03416087157102243)
│  └─ Measure{2,1,AbstractBlock}


In [4]:
function gensample(circuit, basis; nbatch=1024)
    mblocks = collect_blocks(Measure, circuit)
    for m in mblocks
        m.operator = basis
    end
    reg = zero_state(nqubits(circuit); nbatch=nbatch)
    reg |> circuit
    mblocks
end

gensample (generic function with 1 method)

In [5]:
res = gensample(circuit, X; nbatch=1024)

4-element Array{Measure,1}:
 Measure{2,1,AbstractBlock}
 Measure{2,1,AbstractBlock}
 Measure{2,1,AbstractBlock}
 Measure{2,1,AbstractBlock}

In [6]:
res[1].results

1024-element Array{Int64,1}:
 -1
  1
  1
  1
  1
  1
 -1
 -1
 -1
  1
  1
 -1
  1
  ⋮
  1
  1
  1
  1
  1
 -1
  1
  1
  1
  1
  1
  1

In [7]:
struct TFIChain
    length::Int
    h::Float64
    TFIChain(length::Int; h::Real) = new(length, Float64(h))
end

In [8]:
model = TFIChain(4; h=0.5)

TFIChain(4, 0.5)

In [9]:
function energy(circuit, model::TFIChain; nbatch=1024)
    # measuring Z basis
    mblocks = gensample(circuit, Z; nbatch=nbatch)
    local eng = 0.0
    for (a, b, v) in ((i, i+1, 1.0) for i=1:model.length-1)
        eng += v*mean(mblocks[a].results .* mblocks[b].results)
    end
    eng/=4

    # measuring X basis
    mblocks = gensample(circuit, X; nbatch=nbatch)
    engx = sum(mean.([m.results for m in mblocks]))
    eng + model.h*engx/2
end

energy (generic function with 1 method)

In [10]:
energy(circuit, model)

0.27734375

In [11]:
function chem2circuit(circuit)
    nbit = length(collect_blocks(Measure, circuit))
    nm = 1
    nv = 1
    c = chain(nbit)
    for (i, blk) in enumerate(circuit)
        blk = chain([b for b in blk if !(b isa Measure)]...)
        push!(c, concentrate(nbit, blk, [(i-1)*nm+1:i*nm..., nbit-nv+1:nbit...]))
    end
    c
end

chem2circuit (generic function with 1 method)

In [12]:
bigc = chem2circuit(circuit)

nqubits: 4, datatype: Complex{Float64}
chain
├─ Concentrator: (1, 4)
│  └─ chain
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X gate, 0.84570445572993)
│     │     └─ rot(Z gate, 0.4318469396717437)
│     ├─ control(1)
│     │  └─ (2,)X gate
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X gate, 0.6605822615388279)
│     │     └─ rot(Z gate, 0.22101235663174257)
│     ├─ control(1)
│     │  └─ (2,)Z gate
│     └─ put on (1)
│        └─ chain
│           ├─ rot(Z gate, 0.49914112233806573)
│           └─ rot(X gate, 0.04810499026441151)
├─ Concentrator: (2, 4)
│  └─ chain
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X gate, 0.28418220191407983)
│     │     └─ rot(Z gate, 0.9350756615919091)
│     ├─ control(1)
│     │  └─ (2,)X gate
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X gate, 0.9010875863516428)
│     │     └─ rot(Z gate, 0.7208934347561435)
│     ├─ control(1)
│     │  └─ (2,)Z gate
│     └─ put on (1)
│        └─ chain
│          

In [13]:
function hamiltonian(model::TFIChain)
    nbit = model.length
    sum(repeat(nbit, Z, (i,i+1)) for i=1:nbit-1)*0.25 +
    sum(put(nbit, i=>X) for i=1:nbit)*0.5model.h
end

hamiltonian (generic function with 1 method)

In [14]:
h = hamiltonian(model)

nqubits: 4, datatype: Complex{Float64}
sum
├─ [scale: 0.25] sum
│     ├─ repeat on (1, 2)
│     │  └─ Z gate
│     ├─ repeat on (2, 3)
│     │  └─ Z gate
│     └─ repeat on (3, 4)
│        └─ Z gate
└─ [scale: 0.25] sum
      ├─ put on (1)
      │  └─ X gate
      ├─ put on (2)
      │  └─ X gate
      ├─ put on (3)
      │  └─ X gate
      └─ put on (4)
         └─ X gate


In [15]:
expect(h, zero_state(nqubits(bigc)) |> bigc) |> real

0.2757396782114322

In [18]:
function train(circuit, model; maxiter=200, α=0.3, nbatch=1024)
    rots = collect(RotationGate, circuit)
    for i in 1:maxiter
        for r in rots
            r.theta += π/2
            E₊ = energy(circuit, model; nbatch=nbatch)
            r.theta -= π
            E₋ = energy(circuit, model; nbatch=nbatch)
            r.theta += π/2
            g = 0.5(E₊ - E₋)
            r.theta -= g*α
        end
        println("Iter $i, E/N = $(energy(circuit, model, nbatch=nbatch)/model.length)")
    end
    circuit
end

train (generic function with 1 method)

In [19]:
train(circuit, model)

┌ Warning: `collect(t::Type, x::AbstractBlock)` is deprecated, use `collect_blocks(t, x)` instead.
│   caller = #train#20(::Int64, ::Float64, ::Int64, ::Function, ::ChainBlock{2,Complex{Float64}}, ::TFIChain) at In[18]:2
└ @ Main ./In[18]:2


Iter 1, E/N = -0.2568359375
Iter 2, E/N = -0.2574462890625
Iter 3, E/N = -0.2666015625
Iter 4, E/N = -0.266357421875
Iter 5, E/N = -0.262451171875
Iter 6, E/N = -0.263916015625
Iter 7, E/N = -0.264404296875
Iter 8, E/N = -0.2657470703125
Iter 9, E/N = -0.26806640625
Iter 10, E/N = -0.266357421875
Iter 11, E/N = -0.26513671875
Iter 12, E/N = -0.25732421875
Iter 13, E/N = -0.2615966796875
Iter 14, E/N = -0.263427734375
Iter 15, E/N = -0.2657470703125
Iter 16, E/N = -0.259521484375
Iter 17, E/N = -0.268798828125
Iter 18, E/N = -0.26416015625
Iter 19, E/N = -0.261962890625
Iter 20, E/N = -0.26953125
Iter 21, E/N = -0.267822265625
Iter 22, E/N = -0.2694091796875
Iter 23, E/N = -0.2657470703125
Iter 24, E/N = -0.2666015625
Iter 25, E/N = -0.2730712890625
Iter 26, E/N = -0.2628173828125
Iter 27, E/N = -0.265869140625
Iter 28, E/N = -0.26953125
Iter 29, E/N = -0.2685546875
Iter 30, E/N = -0.2686767578125
Iter 31, E/N = -0.2689208984375
Iter 32, E/N = -0.276123046875
Iter 33, E/N = -0.270751953

nqubits: 2, datatype: Complex{Float64}
chain
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 1.6215589479174288)
│  │     └─ rot(Z gate, 1.5887072912342408)
│  ├─ control(1)
│  │  └─ (2,)X gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 1.5326525740388321)
│  │     └─ rot(Z gate, -0.7233968230557567)
│  ├─ control(1)
│  │  └─ (2,)Z gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(Z gate, -0.5611127839119343)
│  │     └─ rot(X gate, 1.5552827246394112)
│  └─ Measure{2,1,AbstractBlock}
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 1.26499274878908)
│  │     └─ rot(Z gate, -1.2865796118455897)
│  ├─ control(1)
│  │  └─ (2,)X gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X gate, 0.5024547738516401)
│  │     └─ rot(Z gate, 0.711152223818645)
│  ├─ control(1)
│  │  └─ (2,)Z gate
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(Z gate, 0.6918468399764044)
│  │     └─ rot(X gate, -0.44821217530397484)
│  └─ Measure{2,1,AbstractBlock}
└

In [21]:
EG = eigen(mat(h) |> Matrix).values[1]
@show EG/nspin

EG / nspin = -0.29742315519647605


-0.29742315519647605

In [23]:
EV = eigen(mat(h) |> Matrix).vectors[:,1]

16-element Array{Complex{Float64},1}:
  0.10878394077769024 + 0.0im
  -0.1666666666666665 + 0.0im
 -0.25534814770632663 + 0.0im
  0.14656420692863603 + 0.0im
  -0.2553481477063265 + 0.0im
   0.4798975402619696 + 0.0im
  0.22454939255564377 + 0.0im
  -0.1666666666666668 + 0.0im
 -0.16666666666666657 + 0.0im
  0.22454939255564235 + 0.0im
  0.47989754026197007 + 0.0im
  -0.2553481477063253 + 0.0im
    0.146564206928636 + 0.0im
  -0.2553481477063251 + 0.0im
  -0.1666666666666669 + 0.0im
  0.10878394077768962 - 0.0im

In [32]:
reg = ArrayReg(EV)

ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 4/4

In [33]:
r = reg |> focus!(1, 2) |> ρ

DensityMatrix{1,Complex{Float64},Array{Complex{Float64},3}}
    active qubits: 2/2

In [39]:
entropy(spectrum) = -sum(spectrum .* log.(spectrum))/log(2)

entropy (generic function with 1 method)

In [40]:
entropy(eigen(r.state[:,:,1]).values)

0.41098665651454297